In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [2]:
NUTRITION_FILE = '../data/nutrition.csv'
PRICING_FILE = '../data/pricing.csv'

In [3]:
nutrition = pd.read_csv(NUTRITION_FILE, index_col=0)
pricing = pd.read_csv(PRICING_FILE, index_col=0)

In [4]:
grouped = (pricing[pricing.country_name == 'Armenia']
                  .groupby(['country_name', 'locality_name', 'market_name']))

In [5]:
solns = {}

for market, idx in grouped.groups.items():
    df = pricing.loc[idx]

    A_ub = -np.transpose(nutrition.values)
    b_ub = -np.array([65, 300, 25, 50])

    # Construct c appropriately (i.e. add 0s to the appropriate foods)
    c = pd.Series(data=np.zeros(84), index=nutrition.index)
    c.loc[df.commodity_name] = df.price.values
    c = c.values

    solns[market] = linprog(c, A_ub, b_ub)

In [6]:
pd.DataFrame(data=[soln.x for soln in solns.values()],
             index=solns.keys(),
             columns=nutrition.index).transpose().to_csv('armenia.csv')